# 코로나 19 확진자 및 예방접종센터, 선별진료소 데이터 마이닝 서비스
## 김지태
## 2023년 4월 기준, 기존 github에 업로드된 url이 변경되어 4월 버전으로 재업로드함.

# 주제 선정 및 이유

최근 코로나 19의 영향력이 계속되는 가운데, 해외출국자, 고령자 등을 위하여 지속적인 예방접종 서비스가 유지되어야 한다.<br>
원활한 예방접종을 위해서는 예방접종을 시행하는 병원, 보건소 등의 위치를 정확히 파악해야 한다.<br>
이에 따라, data.go.kr에서 제공하는 누적 확진자수의 api를 통해 누적 확진, 사망자 현황을 그래프로 제시하고, ncov.mohw.go.kr에서 제공하는 일일확진자, 사망자, 및 재원 위중증자 자료를 크롤링 하여 그래프 정보로 제공할 예정이다.<br>
또한, 시도별 예방접종병원과 선별진료소를 파악하기 위해 카카오 지도 api와 연동하여 그 결과값을 보여줄 예정이다.

# 프로젝트 목표


다음 프로그램을 통하여 일일 및 누적 확진자, 사망자, 위중증자 추이를 알기 쉽게 설정하며, 예방접종센터 및 선별진료소를 각 지역 별로 찾기 쉽도록 하는 것이 이번 프로젝트의 목표이다.<br>
*본 프로그램은 mac(ARM) 환경에서 제작되었으며 동봉된 korea.csv파일과 프로그램이 같은 경로에 있어야 합니다.

# 프로그램 구성

프로그램의 기본 뼈대는 tkinter를 이용한 gui 형태로 제작된다.<br>
우선, 왼쪽 상단에는 일일 확진자수가 표시된다.<br>
확진자수 옆의 '자세히 보기'버튼을 누르면 7일 간의 확진자, 위중증, 사망자 추이를 각각 일일별로, 누적치로 확인해 볼 수 있다.<br>
다음으로 프로그램의 중앙에는 시도와 시군구를 설정할 수 있는 콤보박스와 함께 찾기 버튼이 표시된다. <br>
시도와 시군구를 설정한 뒤 '백신 접종소 찾기' 버튼을 누르면 해당 도시에 맞는 백신 접종소가 하단에 제시된다.<br>
마찬가지로, '선별진료소 찾기' 버튼을 누르면 해당 도시에 맞는 선별진료소가 하단에 제시된다.

# 데이터 활용 측면 분석

이 프로그램에서 데이터를 활용하는 방법은 총 세 가지가 있다.<br> 
첫째로, 콤보박스 활용을 위해 각 지역의 정보를 담은 csv데이터를 활용하여야 한다.<br>
프로그램 메인 화면에서의 콤보박스는 총 세개로 구성되어 있으며, 오른쪽으로 갈 수록 하위 지역에 해당한다. <br>
모든 지역을 일일이 리스트에 작성하기에는 시간이 매우 오래걸리므로, for문과 if문을 적절히 활용하여 지역 데이터를 콤보박스에 옮겨야 한다.<br>
둘째로, beautifulsoup를 이용한 웹사이트 크롤링 작업이 요구된다.<br>
코로나 공식 홈페이지에서 제시된 7일 간의 확진자, 위중증자, 사망자 데이터를 기반으로 matplotlib를 이용하여 시각화된 그래프 자료로 제공할 예정이다.<br>
마지막으로, api를 기반으로 한 프로그래밍 작업이 요구된다.<br>
공공데이터 사이트에서 제공하는 코로나 누적 확진자, 사망자, 위중증자 수를 api를 통하여 추출하며 matplotlib를 통해 가공할 예정이다.<br>
또한, 카카오지도 api를 이용하여 원하는 지역의 백신접종 병원과 선별진료소를 제공할 예정이다.

# Tkinter를 이용한 프로그램 뼈대

다음과 같이 tkinter를 이용하여 프로그램의 기본 뼈대를 만들어 보았다.<br>
추후 함수와 연결할 곳은 각각 소스코드에 대한 부가설명, 혹은 연결해야될 함수는 #으로 표시하였다.<br>
말 그대로 GUI 프로그램의 가장 기본적인 뼈대에 불과하며, 중요한 모듈과 함수는 아직 호출되거나 사용되지 않았다.

In [2]:
import tkinter 
import tkinter.ttk as ttk
window=tkinter.Tk()#메인 창 선언
window.title("코로나19 데이터 분석")#메인 창의 제목
window.geometry("1200x500+100+100")#메인 창의 크기
window.resizable(False, False)#화면 크기 변경 불가능
### 각종 클래스 및 함수 공간


###
state=["서울특별시","경기도","인천광역시","충청남도","충청북도","강원도","경상남도","경상북도","전라남도","전라북도","광주광역시","울산광역시","대전광역시","부산광역시","대구광역시","제주특별자치도","세종특별자치시"]
#광역자치단체 목록
combobox = ttk.Combobox(window)#콤보박스를 표시할 메인창
combobox.config(height=5)#광역단체 combobox          
combobox.config(values=state)           
combobox.config(state="readonly")   
combobox.set("광역자치단체")          
combobox.grid(row=10,column=10)
stateok=ttk.Combobox(window)#기초단체 combobox
stateok.config(height=5)           
stateok.set("고르세요")   
stateok.grid(row=10,column=20)  
stateok.config(values="readonly")
stategu=ttk.Combobox(window)#행정동 combobox
stategu.config(height=5)           
stategu.set("고르세요")   
stategu.grid(row=10,column=30)  
stategu.config(values="readonly")
btn0 = tkinter.Button(window,text="백신\n접종소 찾기", width=10)#이후 command 연결
btn0.grid(row=10,column=40)
btn1 = tkinter.Button(window,text="선별\n진료소 찾기", width=10)#이후 command 연결
btn1.grid(row=10,column=50)
label=tkinter.Label(window, text="일일 \n 확진자수", width=10, height=5)
label.grid(row=0,column=0)
covidlabel=tkinter.Label(window, text="제공예정 확진자수", width=10, height=5)#제공예정 확진자수 이후 확진자 표시 함수와 연결
covidlabel.grid(row=0,column=2)
covidmoreinfo=tkinter.Button(window,text="자세히",width=10,height=5)#이후 command 연결
covidmoreinfo.grid(row=0,column=4)
combobox.bind("<<ComboboxSelected>>")#이후 각각의 for문 함수와 연결
stateok.bind("<<ComboboxSelected>>")#이후 각각의 for문 함수와 연결
stategu.bind("<<ComboboxSelected>>")#이후 콤보박스 전부 합치는 함수와 연결
window.mainloop()

# 메인 화면 확진자수 크롤링 코드

프로그램 메인 창의 뼈대가 완성되었으니, 가장 먼저 메인 화면에 표시될 일일 확진자를 크롤링하는 코드를 제작해야 한다.<br>
requests와 BeautifulSoup를 호출하였으며, 이를 통해 당일 확진자 크롤링 작업을 진행하였다.<br>
url에는 크롤링할 홈페이지 주소를 할당했다. <br>
이후 홈페이지에서 확진자 숫자가 표시된 영역의 selector를 복사하여 특정 영역의 데이터만 호출되도록 설정했다.<br>
covidnum은 최종적으로 표시될 확진자수가 담길 변수이다.

In [6]:
import requests
from bs4 import BeautifulSoup
url = 'http://ncov.kdca.go.kr/' 
covidresponse = requests.get(url) 
html = covidresponse.text
soup = BeautifulSoup(html, 'html.parser')
number_of_corona = soup.select_one('#content > div > div > div.liveboard_layout > div.liveToggleOuter > div > div.live_left > div.occurrenceStatus > div.occur_graph > table > tbody > tr:nth-child(1) > td:nth-child(5) > span') 
covidnum=number_of_corona.get_text()
print(covidnum)

11,666


이렇게 다음과 같이 일일 확진자 숫자가 제대로 표시됨을 알 수 있다.<br>
이를 첫번째 소개된 tkinter 뼈대에 적용해 보자.<br>
covidlabel=tkinter.Label(window, text="제공예정 확진자수", width=10, height=5)#제공예정 확진자수 이후 확진자 표시 함수와 연결
이 부분에서 text="제공예정 확진자수"를 text=covidnum으로 교체하면 된다.

# 전국 광역, 기초, 행정동 엑셀 자료 활용

전국 광역, 기초, 행정동의 자료를 콤보박스에 적용하기 위해서는 적절한 반복문의 사용이 요구된다.<br>
오른쪽의 콤보박스가 왼쪽에 포함되는 관계이므로 조건문의 사용이 요구되기도 한다.<br>
적절한 반복문과 조건문이 사용될 경우 코드의 길이를 기하급수적으로 줄일 수 있다.<br>
반대로, 이를 제대로 활용하지 못할 경우 코드의 길이는 상상을 초월하게 늘어난다.<br>
직접 korea.csv파일을 호출하여 이를 구현해보자.

In [7]:
#광역 to 기초 콤보박스 연동
fileMatrix=[]
stateresult=[]#광역자치단체의 기초자치단체가 들어갈 리스트
with open('korea.csv', 'r',encoding='euc-kr') as file:
    for lineContent in file:
        fileMatrix.append(lineContent.strip('\n').split(','))
    for i in fileMatrix:
        if "서울특별시"==i[1]:#첫 콤보로부터 받은 광역자치단체가 같을 경우(일단은 서울특별시를 예시로, 원래는 combobox.get())
            stateresult.append(i[3])#엑셀 파일의 4번째 열을 리스트의 요소로 추가함
            stateresult=list(set(stateresult))#중복 제거
print(stateresult)

#기초 to 행정동 연동
fileMatrix=[]
guresult=[]#기초자치단체의 행정동이 들어갈 리스트
with open('korea.csv', 'r',encoding='euc-kr') as file:
    for lineContent in file: # Point.1
        fileMatrix.append(lineContent.strip('\n').split(',')) # Point.2
    for i in fileMatrix:
        if "강남구"==i[3]:#두 번째 콤보로부터 받은 기초자치단체가 같을 경우(일단은 강남구를 예시로, 원래는 stateok.get())
            guresult.append(i[5])#엑셀 파일의 6번째 열을 리스트의 요소로 추가함
print(guresult)                


['', '용산구', '노원구', '금천구', '중구', '서대문구', '동대문구', '광진구', '양천구', '강서구', '구로구', '중랑구', '관악구', '마포구', '강동구', '서초구', '영등포구', '강남구', '송파구', '성북구', '동작구', '도봉구', '성동구', '은평구', '강북구', '종로구']
['', '신사동', '논현1동', '논현2동', '압구정동', '청담동', '삼성1동', '삼성2동', '대치1동', '대치2동', '대치4동', '역삼1동', '역삼2동', '도곡1동', '도곡2동', '개포1동', '개포2동', '개포4동', '세곡동', '일원본동', '일원1동', '일원2동', '수서동']


이렇게 정상적으로 리스트가 호출됨을 알 수 있다.<br>
이를 응용하여 프로그램의 뼈대에 실제로 적용해 보자.

In [14]:
import tkinter 
import tkinter.ttk as ttk
import requests
from bs4 import BeautifulSoup
window=tkinter.Tk()#메인 창 선언
window.title("코로나19 데이터 분석")#메인 창의 제목
window.geometry("1200x500+100+100")#메인 창의 크기
window.resizable(False, False)#화면 크기 변경 불가능
### 각종 클래스 및 함수 공간

#메인 화면 확진자수 크롤링 코드
url = 'http://ncov.kdca.go.kr/' 
covidresponse = requests.get(url) 
html = covidresponse.text
soup = BeautifulSoup(html, 'html.parser')
number_of_corona = soup.select_one('#content > div > div > div.liveboard_layout > div.liveToggleOuter > div > div.live_left > div.occurrenceStatus > div.occur_graph > table > tbody > tr:nth-child(1) > td:nth-child(5) > span') 
covidnum=number_of_corona.get_text()
print(covidnum)
###############################콤보박스 활용###################################
#광역 to 기초 콤보박스 연동
def statecombo(a):
    fileMatrix=[]
    stateresult=[]#광역자치단체의 기초자치단체가 들어갈 리스트
    with open('korea.csv', 'r',encoding='euc-kr') as file:
        for lineContent in file:
            fileMatrix.append(lineContent.strip('\n').split(','))
        for i in fileMatrix:
            if combobox.get()==i[1]:#첫 콤보로부터 받은 광역자치단체가 같을 경우
                stateresult.append(i[3])#엑셀 파일의 4번째 열을 리스트의 요소로 추가함
                stateresult=list(set(stateresult))#중복 제거
                stateok.config(values=stateresult)#두 번째 콤보박스의 내용물
#기초 to 행정동 연동
def gucombo(a):
    fileMatrix=[]
    guresult=[]#기초자치단체의 행정동이 들어갈 리스트
    with open('korea.csv', 'r',encoding='euc-kr') as file:
        for lineContent in file: # Point.1
            fileMatrix.append(lineContent.strip('\n').split(',')) # Point.2
        for i in fileMatrix:
            if stateok.get()==i[3]:#두 번째 콤보로부터 받은 기초자치단체가 같을 경우
                guresult.append(i[5])#엑셀 파일의 6번째 열을 리스트의 요소로 추가함
                stategu.config(values=guresult)#마지막 콤보박스의 내용물
   
#세개의 콤보박스 합치기
def bindcombo(a):
    global combostr
    comboresult=[]
    combostr=str(combobox.get())+str(stateok.get())+str(stategu.get())
    comboresult.append(combostr)
    print(comboresult)
    return comboresult
###############################콤보박스 활용###################################

state=["서울특별시","경기도","인천광역시","충청남도","충청북도","강원도","경상남도","경상북도","전라남도","전라북도","광주광역시","울산광역시","대전광역시","부산광역시","대구광역시","제주특별자치도","세종특별자치시"]
#광역자치단체 목록
combobox = ttk.Combobox(window)#콤보박스를 표시할 메인창
combobox.config(height=5)#광역단체 combobox          
combobox.config(values=state)           
combobox.config(state="readonly")   
combobox.set("광역자치단체")          
combobox.grid(row=10,column=10)
stateok=ttk.Combobox(window)#기초단체 combobox
stateok.config(height=5)           
stateok.set("고르세요")   
stateok.grid(row=10,column=20)  
stateok.config(values="readonly")
stategu=ttk.Combobox(window)#행정동 combobox
stategu.config(height=5)           
stategu.set("고르세요")   
stategu.grid(row=10,column=30)  
stategu.config(values="readonly")
btn0 = tkinter.Button(window,text="백신\n접종소 찾기", width=10)#이후 command 연결
btn0.grid(row=10,column=40)
btn1 = tkinter.Button(window,text="선별\n진료소 찾기", width=10)#이후 command 연결
btn1.grid(row=10,column=50)
label=tkinter.Label(window, text="일일 \n 확진자수", width=10, height=5)
label.grid(row=0,column=0)
covidlabel=tkinter.Label(window, text=covidnum, width=10, height=5)#제공예정 확진자수 이후 확진자 표시 함수와 연결
covidlabel.grid(row=0,column=2)
covidmoreinfo=tkinter.Button(window,text="자세히",width=10,height=5)#이후 command 연결
covidmoreinfo.grid(row=0,column=4)
combobox.bind("<<ComboboxSelected>>",statecombo)#콤보박스 선택이 끝날 경우 호출할 함수
stateok.bind("<<ComboboxSelected>>",gucombo)#콤보박스 선택이 끝날 경우 호출할 함수
stategu.bind("<<ComboboxSelected>>",bindcombo)#콤보박스 선택이 끝날 경우 호출할 함수
window.mainloop()

11,666


# 일일 확진자/사망자/위중증자 분석

메인 화면의 코로나 확진자 숫자 옆의 '자세히'버튼을 눌렀을 때 생길 새로운 창을 먼저 제작해야 한다.<br> 
'자세히'버튼과 covidanal 함수를 연동시키면, '자세히' 버튼을 눌렀을 때, 새로운 창이 제시가 된다.<br>
새로운 창에는 일일 or 누적을 선택할 수 있는 콤보박스, 데이터의 종류를 결정할 수 있는 콤보박스, 그리고 확인버튼이 제공된다.

In [16]:
#코로나 1주일간 확진자, 사망자, 위중증자 추이 뼈대   
def covidanal():
    datelist=["일일","누적"]#첫 콤보박스 리스트
    covidatalist=["확진자","사망자","위중증자"]#표시할 데이터 정보 콤보박스
    global datecombo#일일, 누적 코모박스
    global peoplecombo#데이터 정보 콤보박스
    global covidanalysis#새로운 창 뼈대
    covidanalysis=tkinter.Toplevel()
    covidanalysis.title("코로나 확진자 추이 분석")
    covidanalysis.geometry("500x200")
    covidanalysis.resizable(width=False,height=False)
    datecombo=ttk.Combobox(covidanalysis)
    datecombo.config(height=5)           
    datecombo.set("선택")   
    datecombo.grid(row=0,column=10) 
    datecombo.config(values=datelist)
    peoplecombo=ttk.Combobox(covidanalysis)
    peoplecombo.config(height=5)
    peoplecombo.set("선택")
    peoplecombo.grid(row=0,column=15)
    peoplecombo.config(values=covidatalist)
    btn1 = tkinter.Button(covidanalysis,text="확인", width=10)
    btn1.grid(row=0,column=20)
    covidanalysis.mainloop()
    datecombo.bind("<<ComboboxSelected>>")
    peoplecombo.bind("<<ComboboxSelected>>")
print(covidanal())

TclError: can't invoke "bind" command: application has been destroyed

뼈대가 완성되었으니, 우리가 궁극적으로 파악할 자료를 만들어보자.<br>
우선, 확진자 or 사망자 or 위중증자(재원자)에 해당하는 데이터를 리스트에 넣어야 한다.<br>
dev는 복사한 selector 값이다.<br>
우선, 일일 확진자 수가 담긴 리스트를 만들어보자.

In [17]:
#일일 확진자 추이
dev='#content > div > div:nth-child(16) > table > tbody > tr:nth-child(1) > td:nth-child'
peoplist=[]
covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
analresponse=requests.get(covidurl)
covidhtml=analresponse.text
covidsoup=BeautifulSoup(covidhtml,'html.parser')
for i in range(2,9):#일주일간 for문 반복
        covidweek=covidsoup.select_one(dev+'('+str(i)+')')
        covidweek=covidweek.get_text().replace(',','')
        covidweek=int(covidweek)
        peoplist.append(covidweek)
print(peoplist)

[10372, 9553, 4005, 12444, 13926, 11596, 11666]


이렇게 일주일 간의 확진자 숫자 리스트를 확인해 보았다.
사망자 및 위중증자도 비슷한 방식으로 진행할 수 있다.

In [19]:
#일일 사망자 추이
dev='#content > div > div:nth-child(9) > table > tbody > tr:nth-child(1) > td:nth-child'
peoplist=[]
covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
analresponse=requests.get(covidurl)
covidhtml=analresponse.text
covidsoup=BeautifulSoup(covidhtml,'html.parser')
for i in range(2,9):
        covidweek=covidsoup.select_one(dev+'('+str(i)+')')
        covidweek=covidweek.get_text().replace(',','')
        covidweek=int(covidweek)
        peoplist.append(covidweek)
print(peoplist)

[4, 2, 8, 10, 14, 5, 5]


In [21]:
#일일(재원) 위중증자 추이
dev='#content > div > div:nth-child(11) > table > tbody > tr:nth-child(1) > td:nth-child'
peoplist=[]
covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
analresponse=requests.get(covidurl)
covidhtml=analresponse.text
covidsoup=BeautifulSoup(covidhtml,'html.parser')
for i in range(2,9):
        covidweek=covidsoup.select_one(dev+'('+str(i)+')')
        covidweek=covidweek.get_text().replace(',','')
        covidweek=int(covidweek)
        peoplist.append(covidweek)
print(peoplist)

[132, 127, 127, 128, 129, 127, 127]


이렇게 세 가지의 일일 데이터를 우리의 눈으로 확인해 보았다.<br> 
이제 본격적으로 각각의 데이터에 해당하는 그래프를 그려보자.<br>
그래프를 그리기에 앞서, x축과 y축을 정해야 한다.<br>
y축은 당연히 데이터에 해당하며, x축은 날짜에 해당한다.<br>
하지만, 여기에서 문제가 발생한다.<br>
날짜는 유동적이기 때문에 매일 각기 다른 x축을 제공해야 하는 것이다.<br>
이를 해결하기 위해 datetime, relativedelta, pandas를 import한다.


In [22]:
import datetime
from dateutil.relativedelta import relativedelta
import pandas
current_time=datetime.datetime.now().strftime('%Y%m%d')
weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y%m%d')
daterange=pandas.date_range(weekbefore,current_time)
datelist=[]
for i in daterange:
    datelist.append(i)
print(datelist)

[Timestamp('2023-04-08 00:00:00', freq='D'), Timestamp('2023-04-09 00:00:00', freq='D'), Timestamp('2023-04-10 00:00:00', freq='D'), Timestamp('2023-04-11 00:00:00', freq='D'), Timestamp('2023-04-12 00:00:00', freq='D'), Timestamp('2023-04-13 00:00:00', freq='D'), Timestamp('2023-04-14 00:00:00', freq='D')]


이제 그래프를 본격적으로 그려보자.<br>
이를 위해 matplotlib, DataFrame를 호출해야 한다.<br>
또한, platform도 호출해야 하는데, 설치된 운영체제에 따라 글꼴이 다르기 때문이다.<br>
우선은 일일 확진자 추이 그래프를 생성한다.

In [24]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import datetime
from dateutil.relativedelta import relativedelta
import pandas
from pandas import DataFrame
import platform #운영체제 확인
#일일 확진자 추이
dev='#content > div > div:nth-child(16) > table > tbody > tr:nth-child(1) > td:nth-child'
peoplist=[]
covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
analresponse=requests.get(covidurl)
covidhtml=analresponse.text
covidsoup=BeautifulSoup(covidhtml,'html.parser')
for i in range(2,9):
        covidweek=covidsoup.select_one(dev+'('+str(i)+')')
        covidweek=covidweek.get_text().replace(',','')
        covidweek=int(covidweek)
        peoplist.append(covidweek)
#날짜
current_time=datetime.datetime.now().strftime('%Y%m%d')
weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y%m%d')
daterange=pandas.date_range(weekbefore,current_time)
datelist=[]
for i in daterange:
    datelist.append(i)

#일일확진자 그래프 추이
if platform.system()=="Darwin":
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system()=="Windows":
    plt.rc('font', family='NanumGothic')
data1={'날짜':datelist,'데이터':peoplist}
dataframe=DataFrame(data1,columns=['날짜','데이터'])
root=tkinter.Tk()
figure=plt.Figure(figsize=(5,4),dpi=100)
ax2=figure.add_subplot(111)
line1= FigureCanvasTkAgg(figure,root)
line1.get_tk_widget().pack(side=tkinter.LEFT,fill=tkinter.BOTH)
dataframe=dataframe[['날짜','데이터']].groupby('날짜').sum()
dataframe.plot(kind='line',legend=True,ax=ax2,color='r',marker='o',fontsize=10)
ax2.set_title('일일추이')
root.mainloop()

나머지 사망자, 위중증자도 비슷한 방식으로 그래프를 추출할 수 있다.<br>
하지만, 상당한 코드 길이가 요구되므로, class 이용하여 간결하게 표현하고자 한다.

In [29]:
class Covidanalysis:
    def __init__(self,dev):
        self.dev=dev
        
    #일일확진자수 리스트에 넣기
    def graph(self):    
        global peoplist
        peoplist=[]
        covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
        analresponse=requests.get(covidurl)
        covidhtml=analresponse.text
        covidsoup=BeautifulSoup(covidhtml,'html.parser')
        for i in range(2,9):
            covidweek=covidsoup.select_one(self.dev+'('+str(i)+')')
            covidweek=covidweek.get_text().replace(',','')
            covidweek=int(covidweek)
            peoplist.append(covidweek)
        return peoplist

    def covidanalysisdef(self):
    #x축인 시간대 리스트에 넣기 (최근일로부터 일주일)
        current_time=datetime.datetime.now().strftime('%Y%m%d')
        weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y-%m-%d')
        daterange=pandas.date_range(weekbefore,current_time)
        datelist=[]
        for i in daterange:
            datelist.append(i)
    # 그래프
        plt.rcParams['font.family'] = 'AppleGothic'
        data1={'날짜':datelist,'확진자':peoplist}
        dataframe=DataFrame(data1,columns=['날짜','확진자'])
        root=tkinter.Tk()
        figure=plt.Figure(figsize=(5,4),dpi=100)
        ax2=figure.add_subplot(111)
        line1= FigureCanvasTkAgg(figure,root)
        line1.get_tk_widget().pack(side=tkinter.LEFT,fill=tkinter.BOTH)
        dataframe=dataframe[['날짜','확진자']].groupby('날짜').sum()
        dataframe.plot(kind='line',legend=True,ax=ax2,color='r',marker='o',fontsize=10)
        ax2.set_title('일일추이')
        root.mainloop()
injured=Covidanalysis('#content > div > div:nth-child(16) > table > tbody > tr:nth-child(1) > td:nth-child')#확진자
print(injured.covidanalysisdef())
death=Covidanalysis('#content > div > div:nth-child(9) > table > tbody > tr:nth-child(1) > td:nth-child')#사망자
print(death.covidanalysisdef())
serious=Covidanalysis('#content > div > div:nth-child(11) > table > tbody > tr:nth-child(1) > td:nth-child')#위중증자
print(serious.covidanalysisdef())

None
None
None


이제 이를 콤보박스와 연동하여 프로그램의 뼈대에 적용해 보자.

# 누적 확진자수/사망자수 분석 by api

In [31]:
class Covidanalysis:
    def __init__(self,dev):
        self.dev=dev
   #누적데이터 리스트에 넣기
    def nujuk(self):
        global nujuktotal
        current_time=datetime.datetime.now().strftime('%Y%m%d')
        weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y%m%d')
        daterange=pandas.date_range(weekbefore,current_time).strftime('%Y%m%d')#날짜 기간(1주일)
        nujuktotal=[]#데이터 들어갈 리스트 호출
        for i in daterange:
            url = 'http://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api'
            params ={'serviceKey' : 'L2TR/z3TJ5gEArqsHN5wj0D6ZZ/YgfslYTeADrwNufW3TfqZ29/5XsNCyAIOa1lcTD5YCzsxPCmoF7FIcEABXQ==', 'pageNo' : '1', 'numOfRows' : '500', 'apiType' : 'xml', 'status_dt' : i}
            response = requests.get(url, params=params)
            totalsoup=BeautifulSoup(response.text,"html.parser")
            iteritems=totalsoup.find_all("item")
            for i in iteritems:
                death=i.find(self.dev).get_text()
                nujuktotal.append(int(death))
    #그래프 호출
    def covidanalysisdef(self):
    #x축인 시간대 리스트에 넣기 (최근일로부터 일주일)
        current_time=datetime.datetime.now().strftime('%Y%m%d')
        weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y-%m-%d')
        daterange=pandas.date_range(weekbefore,current_time)
        datelist=[]
        for i in daterange:
            datelist.append(i)
    #그래프
        plt.rcParams['font.family'] = 'AppleGothic'
        data1={'날짜':datelist,'확진자':nujuktotal}
        dataframe=DataFrame(data1,columns=['날짜','확진자'])
        root=tkinter.Tk()
        figure=plt.Figure(figsize=(5,4),dpi=100)
        ax2=figure.add_subplot(111)
        line1= FigureCanvasTkAgg(figure,root)
        line1.get_tk_widget().pack(side=tkinter.LEFT,fill=tkinter.BOTH)
        dataframe=dataframe[['날짜','확진자']].groupby('날짜').sum()
        dataframe.plot(kind='line',legend=True,ax=ax2,color='r',marker='o',fontsize=10)
        ax2.set_title('일일추이')
        root.mainloop()
#누적확진자
nujukinjured=Covidanalysis('hpntcnt')
nujukinjured.nujuk()
print(nujukinjured.covidanalysisdef())
#누적사망자
nujukdeath=Covidanalysis('gpntcnt')
nujukdeath.nujuk()
print(nujukdeath.covidanalysisdef())

None


/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


None


이렇게 공공데이터 api를 이용하여 누적 확진자와 사망자 그래프까지 확인할 수 있었다.<br>
현재 공공데이터와 연동할 경우, 단순 웹사이트 크롤링에 비하여 속도가 현저하게 떨어지는 점은 감안해야 한다.<br>

# 카카오지도 api 이용 백신접종가능병원 및 선별진료소 찾기

이제 카카오지도 api를 이용하여 백신접종가능병원 및 선별진료소를 찾아야 한다. <br>
우선은, 병원이름과 주소를 저장할 리스트박스 뼈대부터 만들어야 한다.

In [32]:
window=tkinter.Tk()#메인 창 선언
window.title("코로나19 데이터 분석")#메인 창의 제목
window.geometry("1200x700+100+100")#메인 창의 크기
window.resizable(False, False)#화면 크기 변경 불가능
listbox=tkinter.Listbox(window,selectmode="single",height=30,width=30)
listbox.grid(row=30,column=10)
listbox2=tkinter.Listbox(window,selectmode="single",height=30,width=30)
listbox2.grid(row=30,column=20)
combobox = ttk.Combobox(window)#콤보박스를 표시할 메인창
combobox.config(height=5)#광역단체 combobox          
combobox.config(values=state)           
combobox.config(state="readonly")   
combobox.set("광역자치단체")          
combobox.grid(row=10,column=10)
stateok=ttk.Combobox(window)#기초단체 combobox
stateok.config(height=5)           
stateok.set("고르세요")   
stateok.grid(row=10,column=20)  
stateok.config(values="readonly")
stategu=ttk.Combobox(window)#행정동 combobox
stategu.config(height=5)           
stategu.set("고르세요")   
stategu.grid(row=10,column=30)  
stategu.config(values="readonly")
btn0 = tkinter.Button(window,text="백신\n접종소 찾기", width=10)
btn0.grid(row=20,column=20)
btn1 = tkinter.Button(window,text="선별\n진료소 찾기", width=10)
btn1.grid(row=20,column=30)
label=tkinter.Label(window, text="일일 \n 확진자수", width=10, height=5)
label.grid(row=0,column=0)
covidlabel=tkinter.Label(window, text=covidnum, width=10, height=5)#제공예정 확진자수 이후 확진자 표시 함수와 연결
covidlabel.grid(row=0,column=2)
covidmoreinfo=tkinter.Button(window,text="자세히",width=10,height=5)#이후 command 연결
covidmoreinfo.grid(row=0,column=4)
window.mainloop()

이렇게 뼈대를 만들었으니, 본격적으로 카카오 api를 통해 원하는 위치정보를 추출해내자.

In [33]:
class Locale:
    @staticmethod
    def MakeLocale(post):
        try:
            pn = post['place_name']
            road_address_name=post['road_address_name']
            return Locale(pn,road_address_name)
        except:
            return None
    def __init__(self,pn,road_address_name):
        self.pn = pn
        self.road_address_name=road_address_name
key ='a60a4f60d1290df09686cc6d2d4f060f'
site='https://dapi.kakao.com/v2/local/search/keyword.json'
auth_key="KakaoAK "+ key
auth_header = "Authorization"

def SearchLocale(query):
    query = urllib.parse.quote(query)
    query_str = site+"?"+"query="+query
    request = urllib.request.Request(query_str)
    request.add_header(auth_header,auth_key)
    response = urllib.request.urlopen(request) #웹 서버에 요청
    rescode = response.getcode()
    if(rescode == 200):
        res = response.read().decode('utf-8')
        jres = json.loads(res)
        if jres == None:
            return []
        locales =[]
        for jloc in jres['documents']:
            locale = Locale.MakeLocale(jloc)
            if(locale!=None):
                locales.append(locale)
        return locales
    else:
        return []
localname=[]#위치 이름 리스트
localaddress=[]#해당 주소 리스트
query = input("지역 검색어:")
locales = SearchLocale(query)
for locale in locales:
    localname.append(locale.pn)
    localaddress.append(locale.road_address_name)
print(localname)
print(localaddress)
#input parameter에 
#서울시 강남구 신사동 백신접종가능병원
#서울시 동대문구 회기동 선별진료소
#입력후, 결과 보기

HTTPError: HTTP Error 400: Bad Request

이제 콤보박스와 연동하여 최종적인 프로그램을 완성하자.

# 최종 소스코드

In [ ]:
import tkinter
import tkinter.ttk as ttk
import requests
from bs4 import BeautifulSoup 
import pandas
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import datetime 
from dateutil.relativedelta import relativedelta
import platform #운영체제에 따라 설치된 기본 글꼴이 다르므로 운영체제 파악
import urllib.request
import json
from tkinter import *#END 호출

window=tkinter.Tk()#메인 창 선언
window.title("코로나19 데이터 분석")#메인 창의 제목
window.geometry("1200x700+100+100")#메인 창의 크기
window.resizable(False, False)#화면 크기 변경 불가능
########################################### 각종 클래스 및 함수 공간##################################################

#######################메인 화면 확진자수 크롤링 코드#####################
url = 'http://ncov.kdca.go.kr/' 
covidresponse = requests.get(url) 
html = covidresponse.text
soup = BeautifulSoup(html, 'html.parser')
number_of_corona = soup.select_one('#content > div > div > div.liveboard_layout > div.liveToggleOuter > div > div.live_left > div.occurrenceStatus > div.occur_graph > table > tbody > tr:nth-child(1) > td:nth-child(5) > span') 
covidnum=number_of_corona.get_text()
#######################메인 화면 확진자수 크롤링 코드#####################
###############################콤보박스 활용###################################
#광역 to 기초 콤보박스 연동
def statecombo(a):
    fileMatrix=[]
    stateresult=[]#광역자치단체의 기초자치단체가 들어갈 리스트
    with open('korea.csv', 'r',encoding='euc-kr') as file:
        for lineContent in file:
            fileMatrix.append(lineContent.strip('\n').split(','))
        for i in fileMatrix:
            if combobox.get()==i[1]:#첫 콤보로부터 받은 광역자치단체가 같을 경우
                stateresult.append(i[3])#엑셀 파일의 4번째 열을 리스트의 요소로 추가함
                stateresult=list(set(stateresult))#중복 제거
                stateok.config(values=stateresult)#두 번째 콤보박스의 내용물
#기초 to 행정동 연동
def gucombo(a):
    fileMatrix=[]
    guresult=[]#기초자치단체의 행정동이 들어갈 리스트
    with open('korea.csv', 'r',encoding='euc-kr') as file:
        for lineContent in file: # Point.1
            fileMatrix.append(lineContent.strip('\n').split(',')) # Point.2
        for i in fileMatrix:
            if stateok.get()==i[3]:#두 번째 콤보로부터 받은 기초자치단체가 같을 경우
                guresult.append(i[5])#엑셀 파일의 6번째 열을 리스트의 요소로 추가함
                stategu.config(values=guresult)#마지막 콤보박스의 내용물
   
#세개의 콤보박스 합치기
def bindcombo(a):
    global combostr
    comboresult=[]
    combostr=str(combobox.get())+str(stateok.get())+str(stategu.get())
    comboresult.append(combostr)
    return comboresult
###############################콤보박스 활용###################################
############################그래프 class#####################################
class Covidanalysis:
    def __init__(self,dev):
        self.dev=dev
   #누적데이터 리스트에 넣기
    def nujuk(self):
        global nujuktotal
        current_time=datetime.datetime.now().strftime('%Y%m%d')
        weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y%m%d')
        daterange=pandas.date_range(weekbefore,current_time).strftime('%Y%m%d')
        nujuktotal=[]
        for i in daterange:
            url = 'http://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api'
            params ={'serviceKey' : 'L2TR/z3TJ5gEArqsHN5wj0D6ZZ/YgfslYTeADrwNufW3TfqZ29/5XsNCyAIOa1lcTD5YCzsxPCmoF7FIcEABXQ==', 'pageNo' : '1', 'numOfRows' : '500', 'apiType' : 'xml', 'status_dt' : i}
            response = requests.get(url, params=params)
            totalsoup=BeautifulSoup(response.text,"html.parser")
            iteritems=totalsoup.find_all("item")
            for i in iteritems:
                death=i.find(self.dev).get_text()
                nujuktotal.append(int(death))

    #일일데이터 리스트에 넣기
    def graph(self):    
        global peoplist
        peoplist=[]
        covidurl='https://ncov.kdca.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
        analresponse=requests.get(covidurl)
        covidhtml=analresponse.text
        covidsoup=BeautifulSoup(covidhtml,'html.parser')
        for i in range(2,9):
            covidweek=covidsoup.select_one(self.dev+'('+str(i)+')')
            covidweek=covidweek.get_text().replace(',','')
            covidweek=int(covidweek)
            peoplist.append(covidweek)
        return peoplist

    def covidanalysisdef(self):
    #x축인 시간대 리스트에 넣기 (최근일로부터 일주일)
        current_time=datetime.datetime.now().strftime('%Y%m%d')
        weekbefore=(datetime.datetime.now()-relativedelta(weeks=1)+relativedelta(days=1)).strftime('%Y-%m-%d')
        daterange=pandas.date_range(weekbefore,current_time)
        datelist=[]
        for i in daterange:
            datelist.append(i)
    #그래프
        if platform.system()=="Darwin":
            plt.rcParams['font.family'] = 'AppleGothic'
        elif platform.system()=="Windows":
            plt.rc('font', family='NanumGothic')
        if datecombo.get()=="일일":
            data1={'날짜':datelist,'인명피해':peoplist}
        elif datecombo.get()=="누적":
            data1={'날짜':datelist,'인명피해':nujuktotal}
        dataframe=DataFrame(data1,columns=['날짜','인명피해'])
        root=tkinter.Tk()
        figure=plt.Figure(figsize=(5,4),dpi=100)
        ax2=figure.add_subplot(111)
        line1= FigureCanvasTkAgg(figure,root)
        line1.get_tk_widget().pack(side=tkinter.LEFT,fill=tkinter.BOTH)
        dataframe=dataframe[['날짜','인명피해']].groupby('날짜').sum()
        dataframe.plot(kind='line',legend=True,ax=ax2,color='r',marker='o',fontsize=10)
        if datecombo.get()=="일일" and peoplecombo.get()=="확진자":
            whichdata='확진자 일일추이'
        elif datecombo.get()=="일일" and peoplecombo.get()=="사망자":
            whichdata='사망자 일일추이'
        elif datecombo.get()=="일일" and peoplecombo.get()=="위중증자":
            whichdata='위중증자 일일추이'
        elif datecombo.get()=="누적" and peoplecombo.get()=="확진자":
            whichdata='누적 확진자 추이'
        elif datecombo.get()=="누적" and peoplecombo.get()=="사망자":
            whichdata='누적 사망자 추이'
        ax2.set_title(whichdata)
        root.mainloop()
############################그래프 class#####################################
###########################자세히 부분 콤보박스-그래프와 연동#######################
def lastchoice():
    if datecombo.get()=="일일" and peoplecombo.get()=="확진자":#확진자
        a=Covidanalysis('#content > div > div:nth-child(16) > table > tbody > tr:nth-child(1) > td:nth-child')
        a.graph()
        print(a.covidanalysisdef())
    elif datecombo.get()=="일일" and peoplecombo.get()=="사망자":#사망자
        b=Covidanalysis('#content > div > div:nth-child(9) > table > tbody > tr:nth-child(1) > td:nth-child')
        b.graph()
        print(b.covidanalysisdef())
    elif datecombo.get()=="일일" and peoplecombo.get()=="위중증자":#위중증자
        c=Covidanalysis('#content > div > div:nth-child(11) > table > tbody > tr:nth-child(1) > td:nth-child')
        c.graph()
        print(c.covidanalysisdef())
    elif datecombo.get()=="누적" and peoplecombo.get()=="확진자":
        d=Covidanalysis('hpntcnt')
        d.nujuk()
        print(d.covidanalysisdef())
    elif datecombo.get()=="누적" and peoplecombo.get()=="사망자":
        e=Covidanalysis('gpntcnt')
        e.nujuk()
        print(e.covidanalysisdef())
    
###########################자세히 부분 콤보박스-그래프와 연동#######################

#################코로나 1주일간 확진자, 사망자, 위중증자 추이 새창 뼈대 #####################  
def covidanal():
    datelist=["일일","누적"]#첫 콤보박스 리스트
    covidatalist=["확진자","사망자","위중증자"]#표시할 데이터 정보 콤보박스
    global datecombo#일일, 누적 코모박스
    global peoplecombo#데이터 정보 콤보박스
    global covidanalysis#새로운 창 뼈대
    covidanalysis=tkinter.Toplevel()
    covidanalysis.title("코로나 확진자 추이 분석")
    covidanalysis.geometry("500x50")
    covidanalysis.resizable(width=False,height=False)
    datecombo=ttk.Combobox(covidanalysis)
    datecombo.config(height=5)           
    datecombo.set("선택")   
    datecombo.grid(row=0,column=10) 
    datecombo.config(values=datelist)
    peoplecombo=ttk.Combobox(covidanalysis)
    peoplecombo.config(height=5)
    peoplecombo.set("선택")
    peoplecombo.grid(row=0,column=15)
    peoplecombo.config(values=covidatalist)
    btn1 = tkinter.Button(covidanalysis,text="확인", width=10,command=lastchoice)#lastchoice 함수와 연동하여 그래프 호출
    btn1.grid(row=0,column=20)
    covidanalysis.mainloop()
    datecombo.bind("<<ComboboxSelected>>")
    peoplecombo.bind("<<ComboboxSelected>>")
#################코로나 1주일간 확진자, 사망자, 위중증자 추이 새창 뼈대 #####################  
####################콤보박스 선택된 지역 병원 호출###################################
class Locale:
    @staticmethod
    def MakeLocale(post):
        try:
            pn = post['place_name']
            road_address_name=post['road_address_name']
            return Locale(pn,road_address_name)
        except:
            return None
    def __init__(self,pn,road_address_name):
        self.pn = pn
        self.road_address_name=road_address_name

key ='a60a4f60d1290df09686cc6d2d4f060f'
site='https://dapi.kakao.com/v2/local/search/keyword.json'
auth_key="KakaoAK "+ key
auth_header = "Authorization"

def SearchLocale(query):
    query = urllib.parse.quote(query)
    query_str = site+"?"+"query="+query
    request = urllib.request.Request(query_str)
    request.add_header(auth_header,auth_key)
    response = urllib.request.urlopen(request) #웹 서버에 요청
    rescode = response.getcode()
    if(rescode == 200):
        res = response.read().decode('utf-8')
        jres = json.loads(res)
        if jres == None:
            return []
        locales =[]
        for jloc in jres['documents']:
            locale = Locale.MakeLocale(jloc)
            if(locale!=None):
                locales.append(locale)
        return locales
    else:
        return []

def vaccine(): #백신접종가능병원 버튼과 연동
    vaccinename=[]
    vaccineaddress=[]
    query = combostr+" 백신접종가능병원"
    locales = SearchLocale(query)
    listbox.delete(0, END)#우선은 리스트 박스 초기화
    listbox2.delete(0, END)#우선은 리스트 박스 초기화
    for locale in locales:
        vaccinename.append(locale.pn)
        vaccineaddress.append(locale.road_address_name)
    for i in vaccinename:
        listbox.insert(vaccinename.index(i),i)#리스트박스에 삽입
    for j in vaccineaddress:
        listbox2.insert(vaccineaddress.index(j),j)#리스트박스에 삽입

def hospital():#선별진료소 버튼과 연동
    hospitalname=[]
    hospitaladdress=[]
    query = combostr+" 선별진료소"
    locales = SearchLocale(query)
    listbox.delete(0, END)#우선은 리스트 박스 초기화
    listbox2.delete(0, END)#우선은 리스트 박스 초기화
    for locale in locales:
        hospitalname.append(locale.pn)
        hospitaladdress.append(locale.road_address_name)
    for i in hospitalname:
        listbox.insert(hospitalname.index(i),i)#리스트박스에 삽입
    for j in hospitaladdress:
        listbox2.insert(hospitaladdress.index(j),j)#리스트박스에 삽입
####################콤보박스 선택된 지역 병원 호출###################################
########################################### 각종 클래스 및 함수 공간##################################################
state=["서울특별시","경기도","인천광역시","충청남도","충청북도","강원도","경상남도","경상북도","전라남도","전라북도","광주광역시","울산광역시","대전광역시","부산광역시","대구광역시","제주특별자치도","세종특별자치시"]
#광역자치단체 목록
listbox=tkinter.Listbox(window,selectmode="single",height=30,width=30)
listbox.grid(row=30,column=10)
listbox2=tkinter.Listbox(window,selectmode="single",height=30,width=30)
listbox2.grid(row=30,column=20)
combobox = ttk.Combobox(window)#콤보박스를 표시할 메인창
combobox.config(height=5)#광역단체 combobox          
combobox.config(values=state)           
combobox.config(state="readonly")   
combobox.set("광역자치단체")          
combobox.grid(row=10,column=10)
stateok=ttk.Combobox(window)#기초단체 combobox
stateok.config(height=5)           
stateok.set("고르세요")   
stateok.grid(row=10,column=20)  
stateok.config(values="readonly")
stategu=ttk.Combobox(window)#행정동 combobox
stategu.config(height=5)           
stategu.set("고르세요")   
stategu.grid(row=10,column=30)  
stategu.config(values="readonly")
btn0 = tkinter.Button(window,text="백신\n접종소 찾기", width=10,command=vaccine)
btn0.grid(row=20,column=20)
btn1 = tkinter.Button(window,text="선별\n진료소 찾기", width=10,command=hospital)
btn1.grid(row=20,column=30)
label=tkinter.Label(window, text="일일 \n 확진자수", width=10, height=5)
label.grid(row=0,column=0)
covidlabel=tkinter.Label(window, text=covidnum, width=10, height=5)#제공예정 확진자수 이후 확진자 표시 함수와 연결
covidlabel.grid(row=0,column=2)
covidmoreinfo=tkinter.Button(window,text="자세히",width=10,height=5,command=covidanal)#이후 command 연결
covidmoreinfo.grid(row=0,column=4)
combobox.bind("<<ComboboxSelected>>",statecombo)#콤보박스 선택이 끝날 경우 호출할 함수
stateok.bind("<<ComboboxSelected>>",gucombo)#콤보박스 선택이 끝날 경우 호출할 함수
stategu.bind("<<ComboboxSelected>>",bindcombo)#콤보박스 선택이 끝날 경우 호출할 함수
window.mainloop()

None


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/1n/_4syj2z53cl7m4y85nqgcfrm0000gn/T/ipykernel_82568/3381272626.py", line 186, in covidanal
    datecombo.bind("<<ComboboxSelected>>")
  File "/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1392, in bind
    return self._bind(('bind', self._w), sequence, func, add)
  File "/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1349, in _bind
    return self.tk.call(what + (sequence,))
_tkinter.TclError: can't invoke "bind" command: application has been destroyed


여기까지 최종적인 프로그램이 완료되었다.<br>
이렇게, 웹사이트 크롤링, 엑셀 데이터 연동, api 호출 세 가지의 측면에서 데이터를 추출하는 방법을 알게 되었고,<br>
이를 통해 코로나 관련 데이터를 분석할 수 있는 프로그램을 완성할 수 있었다.